<a href="https://colab.research.google.com/github/parthag1201/FirstCrew/blob/main/CalciCrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the dependencies
!pip install crewai langchain-community pydantic
!pip install 'crewai[tools]'
!pip install 'crewai[CrewStructuredTool]'
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:0

In [2]:
# Using required packages

from crewai import Crew, Task, Agent, Process, LLM
from pydantic import BaseModel
from crewai.tools.structured_tool import CrewStructuredTool
from crewai.tools import tool
from langchain_community.tools import BraveSearch
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import Ollama  # If we use Ollama instead of Gemini
from google.colab import userdata # For API Secret

In [4]:
# Choose LLM

llm = LLM(
    model='gemini/gemini-2.0-flash',  # Use 'vertex_ai' as the provider for Google's Vertex AI
    api_key=userdata.get('API_KEY')
)

In [23]:
# Define Web Search Engine

@tool("duckduckgo_search_tool")
def duckduckgo_search_wrapper(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun(region="pt-br")
    result = search_tool.run(query)
    return result

In [24]:
product_manager_agent = Agent(
    role="Product Visionary",
    goal=(
        "To define the product vision, strategy, and roadmap for the {topic}. Oversee market research, competitor analysis, "
        "and feature prioritization to ensure alignment with business objectives."
    ),
    backstory=(
        "You are a seasoned product manager in software development with experience in leading cross-functional teams. Your expertise "
        "lies in crafting compelling product strategies that drive market success and deliver sde products."
    ),
    llm=llm,
    verbose=True
)

business_analyst_agent = Agent(
    role="Business Strategist",
    goal=(
        "To gather and analyze business requirements, translate them into technical specifications, on the {topic} and ensure "
        "that solutions meet market needs."
    ),
    backstory=(
        "You are a skilled business analyst with a deep understanding of market dynamics and technical "
        "requirements. Your expertise helps bridge the gap between business stakeholders and developers."
    ),
    llm=llm,
    verbose=True
)

software_architect_agent = Agent(
    role="Technical Architect",
    goal=(
        "To design the overall structure of the {topic} software system, ensuring scalability, performance, and security. "
        "Provide technical guidance and oversight throughout the development lifecycle."
    ),
    backstory=(
        "A seasoned software architect with expertise in designing robust and scalable software systems. "
        "You lead technical teams in implementing best practices and ensuring high-quality code."
        "You are skilled in frontend and backend development of software products"
    ),
    llm=llm,
    verbose=True
)

development_team_lead_agent = Agent(
    role="Technical Lead",
    goal=(
        "To guide the development team in translating requirements into high-quality code. Ensure code quality, "
        "provide mentorship, and solve technical challenges."
        "Code the {topic}"
    ),
    backstory=(
        "You are an experienced technical lead with a strong background in software development. Your role is "
        "to mentor developers and ensure that all code aligns with project guidelines."
    ),
    llm=llm,
    verbose=True
)

ux_ui_designer_agent = Agent(
    role="User Experience Designer",
    goal=(
        "To craft user interfaces for {topic} that are aesthetically pleasing and user-friendly. Ensure that the design aligns "
        "with the product's vision and user needs."
    ),
    backstory=(
        "A skilled UX/UI designer with expertise in creating intuitive and engaging user experiences. Your work "
        "focuses on making software applications both visually appealing and functional."
    ),
    llm=llm,
    verbose=True
)

manager_agent = Agent(
    role="Workflow Maestro",
    goal=(
        "To coordinate agents, manage task dependencies, and ensure all outputs meet quality standards. Your focus "
        "is on delivering a cohesive final product through efficient task management."
    ),
    backstory=(
        "A former project manager with a passion for efficient teamwork, you ensure every process runs smoothly, "
        "overseeing tasks and verifying results."
    ),
    llm=llm,
    verbose=True
)

In [25]:
project_management_task = Task(
    description=(
        "Conduct market research to identify key trends and competitors in the software industry on the {topic}. Focus on "
        "emerging technologies and market needs."
        "Oversee the entire project lifecycle, manage timelines, allocate resources, and ensure projects are "
        "completed on time and within budget."
    ),
    expected_output=(
        "A comprehensive report detailing market trends, competitors, and emerging technologies on the {topic}."
        "Regular project status updates and a final project report on the {topic}."
    ),
)

product_requirements_task = Task(
    description=(
        "Gather and document business and technical requirements for the software product : {topic}. Ensure alignment with "
        "market research findings."
    ),
    expected_output=(
        "A detailed document outlining product requirements, including user stories and technical specifications on the software product : {topic}."
    ),
)

software_architecture_task = Task(
    description=(
        "Design the overall architecture of the software system, ensuring scalability, performance, and security on the software product : {topic}."
    ),
    expected_output=(
        "A detailed architecture document outlining system components, interactions, and technical specifications on the software product : {topic}."
    ),
)

development_task = Task(
    description=(
        "Implement the software product based on the defined architecture and requirements. Ensure high-quality "
        "code and adherence to coding standards."
    ),
    expected_output=(
        "A fully functional software product with well-documented code and tests."
    ),
)

ux_ui_design_task = Task(
    description=(
        "Create user interfaces that are intuitive and visually appealing on the software product : {topic}. Ensure design aligns with product vision "
        "and user needs."
    ),
    expected_output=(
        "Design prototypes and specifications for user interfaces on the software product : {topic}."
    ),
)

workflow_coordination_task = Task(
    description=(
        "Coordinate agents to ensure the development of a basic calculator webpage using HTML, CSS, and JavaScript. "
        "Manage task dependencies and ensure all outputs meet quality standards."
    ),
    expected_output=(
        "A fully functional calculator webpage with a clear workflow report detailing task completion and any challenges faced."
    ),
)

In [26]:
crew = Crew(
    agents=[
        product_manager_agent, business_analyst_agent, software_architect_agent,
        development_team_lead_agent, ux_ui_designer_agent,
    ],
    tasks=[
        project_management_task, product_requirements_task, software_architecture_task, development_task,
        ux_ui_design_task, project_management_task,
    ],
    process=Process.hierarchical,
    manager_agent=manager_agent,
    verbose=True
)


In [27]:
# Start Crew.ai by giving the required input

crew_output = crew.kickoff(inputs={"topic": "Creating a Basic Calculator Webpage using HTML, CSS, and JavaScript"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 27cc45d8-1cc8-43d7-bc64-b64784303931                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress

# Agent: Workflow Maestro
## Task: Conduct market research to identify key trends and competitors in the software industry on the Creating a Basic Calculator Webpage using HTML, CSS, and JavaScript. Focus on emerging technologies and market needs.Oversee the entire project lifecycle, manage timelines, allocate resources, and ensure projects are completed on time and within budget.


🚀 Crew: crew
└── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Workflow Maestro
## Final Answer: 
## Basic Calculator Webpage Development Report

**1. Introduction**

This report outlines the development strategy for a basic calculator webpage, integrating market research findings with technical implementation guidelines. The goal is to create a user-friendly, accessible, and responsive calculator that meets current market expectations and adheres to best practices.

**2. Market Analysis and User Expectations**

**2.1 Key Market Trends:**

*   **Mobile-First Approach:** A significant portion of users access the internet via mobile devices. Therefore, the calculator must be fully responsive and optimized for mobile use.
*   **Accessibility:** Users expect web applications to be accessible, regardless of disabilities. Adherence to WCAG guidelines is crucial.
*   **Progressive Web App (PWA) Features:** Users appreciate the ability to install web applications on their devices for offline access and a native-like experience.
*   **Clean and 

🚀 Crew: crew
├── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (15)
│   ├── 🤖 Agent: Business Strategist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Ask question to coworker (7)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (16)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: e1fa2991-c0c0-4fcf-9c9b-c63182df16c0
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (8)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (17)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 9e6a2d65-3fa1-43d2-8cc0-853770a3eb4e
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (9)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: d82c95e9-67c2-4e29-a67e-b5bc569fdb79
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (19)
│   ├── 🤖 Agent: User Experience Designer
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (20)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (21)
│   ├── 🤖 Agent: Technical Lead
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (22)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (23)
│   ├── 🤖 Agent: User Experience Designer
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (24)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 6554590e-e7cb-4edb-884d-79e170f81c89
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (10)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
       Status: Executing Task...
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (25)
    ├── 🤖 Agent: Workflow Maestro
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (26)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (15)
│   ├── 🤖 Agent: Business Strategist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Ask question to coworker (7)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (16)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: e1fa2991-c0c0-4fcf-9c9b-c63182df16c0
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (8)
│   ├── 🤖 Agent: Product Visionary
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (17)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 9e6a2d65-3fa1-43d2-8cc0-853770a3eb4e
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (9)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: d82c95e9-67c2-4e29-a67e-b5bc569fdb79
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (19)
│   ├── 🤖 Agent: User Experience Designer
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (20)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (21)
│   ├── 🤖 Agent: Technical Lead
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (22)
│   ├── 🤖 Agent: Technical Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (23)
│   ├── 🤖 Agent: User Experience Designer
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (24)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 6554590e-e7cb-4edb-884d-79e170f81c89
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (10)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9
       Status: Executing Task...
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (25)
    ├── 🤖 Agent: Workflow Maestro
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (26)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4d6528e3-7b76-4aaf-ba4a-3c86c5bb09c9                                                                     │
│  Agent: Workflow Maestro                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 27cc45d8-1cc8-43d7-bc64-b64784303931                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [28]:
# Output

print(crew_output)

## Basic Calculator Webpage Development Report

**1. Introduction**

This report outlines the development strategy for a basic calculator webpage, integrating market research findings with technical implementation guidelines. The goal is to create a user-friendly, accessible, and responsive calculator that meets current market expectations and adheres to best practices.

**2. Market Analysis and User Expectations**

**2.1 Key Market Trends:**

*   **Mobile-First Approach:** A significant portion of users access the internet via mobile devices. Therefore, the calculator must be fully responsive and optimized for mobile use.
*   **Accessibility:** Users expect web applications to be accessible, regardless of disabilities. Adherence to WCAG guidelines is crucial.
*   **Progressive Web App (PWA) Features:** Users appreciate the ability to install web applications on their devices for offline access and a native-like experience.
*   **Clean and Intuitive Design:** The calculator interface s